In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
df_train = pd.read_csv('data/train.txt')
df_train.keys()

Index([u'Timestamp (DHHMM)', u' S1', u' S2', u' S3', u' S4', u' S5', u' S6',
       u' S7', u' S8', u' S9', u' S10', u' S11', u' S12', u' S13', u' S14',
       u' S15', u' S16', u' S17', u' S18', u' S19', u' S20', u' S21', u' S22',
       u' S23', u' S24', u' S25', u' S26', u' S27', u' S28', u' S29', u' S30',
       u' S31', u' S32', u' S33', u' S34', u' S35', u' S36', u' S37', u' S38',
       u' S39', u' S40', u' S41', u' S42', u' S43', u' S44', u' S45', u' S46',
       u' S47', u' S48', u' S49', u' S50', u' S51', u' S52', u' S53', u' S54',
       u' S55', u' S56'],
      dtype='object')

In [6]:
df_train.sort(u'Timestamp (DHHMM)')
df_train.head()

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Timestamp (DHHMM),S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S47,S48,S49,S50,S51,S52,S53,S54,S55,S56
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# converts time stamp as int in DHHMM into index for the table
def timestamptoindex(ts):
    mm = ts % 100
    hh = (ts % (100 * 100)) // 100
    d = ts // (100 * 100)
    
    index = mm + 60 * hh + 60 * 24 * (d - 1)
    return index

In [24]:
timestamptoindex(10000)

0

In [26]:
# no exception means, functions works alright!
for i, row in df_train.iterrows():
    assert timestamptoindex(row[u'Timestamp (DHHMM)']) == i